In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
mapp = pd.read_csv("emnist/emnist-letters-mapping.txt", delimiter = ' ', \
                   index_col=0, header=None, squeeze=True)

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    import gzip
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte.gz'% kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte.gz'% kind)
    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,offset=8)
    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels

In [1]:
#Load tflite model and initialize interpreter
dataset = 'emnist' # or 'emnist'
output_layers = 10 # default, will be overridden

In [5]:
emnist_distinct_upper = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'N', 'Q', 'R', 'T'] # Characters with distinct uppercase
emnist_nondistinct_upper = ['c', 'i', 'j', 'k', 'l', 'm', 'o', 'p', 's', 'u', 'v', 'w', 'x', 'y', 'z'] # Characters with non-distinct uppercase
emnist_nondistinct_upper_partial = ['c', 'i', 'k', 'l', 'm', 'o', 'p', 's', 'v', 'x', 'z'] # A selection of 11 characters to use as our model can not be too big

In [31]:
                         # always uppercase  # Choose one of ('emnist_distinct_upper', 'emnist_nondistinct_upper_partial')
emnist_characters = [ord(i.upper()) for i in emnist_distinct_upper]

In [33]:
                                    # COLUMN 1 is uppercase
emnist_characters_ascii = list(mapp[mapp[1].isin(emnist_characters)].index)

In [34]:
if dataset == 'mnist':
    output_layers = 10
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
elif dataset == 'emnist':
    output_layers = 11
    x_test, y_test = load_mnist('emnist', kind='emnist-letters-test')
    mask = np.isin(y_test, emnist_characters_ascii)
    x_test = x_test[mask]
    y_test = y_test[mask] - 1

In [2]:
x_test = x_test / 255.0
x_test = x_test.reshape(x_test.shape[0],28,28,1)

print(x_test.shape)

interpreter = tf.lite.Interpreter(model_path=dataset + "-quantized_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors. (can be used to debug any problems)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# print(interpreter.get_tensor_details())

# Example on how you can use this to debug
# print(output_details[0])

# This will output something like this:
#{'name': 'Identity', 'index': 10, 'shape': array([ 1, 10]), 'shape_signature': array([ 1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


# # Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# #invoking the interpreterd, followed by printing the predicted result
interpreter.invoke()
# print(interpreter.get_tensor(output_details[0]['index']))

#predict every row from x_test
lst_results = []
for row in x_test:
    x = tf.expand_dims(np.array(row, dtype=np.float32),0)
    interpreter.set_tensor(input_details[0]['index'], x)
    interpreter.invoke()
    lst_results.append(np.argmax(interpreter.get_tensor(output_details[0]['index'])))
    
#Wrap up by calculating the accuracy
correct = 0
for i in range(len(y_test)):
    if y_test[i] == lst_results[i]:
        correct += 1
        

(8000, 28, 28, 1)


In [3]:
print(f"Correct: {correct}") 
print(f"Total smaples: {len(y_test)}") 
print(f"Accuracy: {correct/len(y_test)}")

Correct: 7130
Total smaples: 8000
Accuracy: 0.89125


In [21]:
lower = np.random.randint(len(x_test))

In [23]:
lower = np.random.randint(len(x_test))
for i in range(lower, lower + 9):
    plt.subplot(330 + (i+1))
    plt.imshow(x_test[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
    plt.title(chr(mapp.iloc[y_test[i]][1]))

ValueError: Integer subplot specification must be a three-digit number, not 3870

<Figure size 432x288 with 0 Axes>

In [10]:
mapp_values = mapp.values

In [16]:
mapp.iloc[0]

1    65
2    97
Name: 1, dtype: int64